In [ ]:
import numpy as np
import pandas as pd
import math
import joblib

from scripts.mechanics_reduction import map_mechanics_to_categories
from scripts.mechanics_reduction import MechanicClustering
from scripts.average_rating_conversion import clean_rating_column
from scripts.time_conversion import convert_publish_year
from scripts.time_conversion import convert_playtime_column
from scripts.outliers_removal import clean_min_player
from scripts.outliers_removal import clean_max_player

# Data Cleaning
## Loading the necessary data and models

In [ ]:
df = pd.read_excel('./data/BGG_Data_Set.xlsx')

In [ ]:
domain_imputing_model = joblib.load('./models/saved/domain_imputing/Multi_Output_LightGBM.pkl')
mechanics_imputing_model = joblib.load('./models/saved/mechanics_imputing/multi_output_classifier_weighted_custom.pkl')

# Duplicates
#### Dropping possible duplicates

In [ ]:
print(f"Number of rows before dropping duplicates: {df.shape[0]}")
print(f"Number of duplicates: {df.duplicated().sum()}")
df = df.drop_duplicates()
print(f"Number of rows after dropping duplicates: {df.shape[0]}")

# Irrelevant columns
#### Dropping irrelevant columns

In [ ]:
df = df.drop(columns=['ID', 'Name', 'BGG Rank'])

# Converting non-numerical Columns
#### Identifying possible non-numerical columns

In [ ]:
df.dtypes

#### Converting the year published column

In [ ]:
df = convert_publish_year(df, 'Year Published')

#### Converting the playtime column

In [ ]:
df = convert_playtime_column(df, 'Play Time')

#### Converting the average rating column

In [ ]:
df = clean_rating_column(df, 'Rating Average')

# Handling outliers
#### Checking for possible outliers

In [ ]:
df.describe()

#### Handling outliers in the average minimum players column

In [ ]:
print(f"Number of rows before cleaning the Min Players column: {df.shape[0]}")
df = clean_min_player(df, column_name='Min Players')
print(f"Number of rows after cleaning the Min Players column: {df.shape[0]}")

#### Handling outliers in the maximum players column

In [ ]:
print(f"Number of rows before cleaning the Max Players column: {df.shape[0]}")
df = clean_max_player(df, column_name='Max Players', compare_column='Min Players')
print(f"Number of rows after cleaning the Max Players column: {df.shape[0]}")

# Handling missing values
#### Checking for missing values

In [ ]:
df.isna().sum().sort_values(ascending=False)

#### Checking the percentage of missing values for each column

In [ ]:
# Calculate the percentage of missing values for each column
missing_percentage = round(df.isna().sum() * 100 / len(df))

# Print the percentage of missing values for each column
print("Percentage of missing values for each column:")
missing_percentage.sort_values(ascending=False)

#### Dropping rows with missing values for the Owned Users column

In [ ]:
print(f"Number of rows before dropping missing values: {df.shape[0]}")
print(f"Number of missing values in the Owned Users column: {df['Owned Users'].isna().sum()}")
df = df.dropna(subset=['Owned Users'])
print(f"Number of rows after dropping missing values: {df.shape[0]}")

#### Dropping rows with missing values for the Play Time and Rating Average columns

In [ ]:
# df = df.dropna(subset=['Play Time', 'Rating Average'], how='all')

#### Dropping the Year Published column because of the low correlation & low information gain with the target variables

In [ ]:
df = df.drop(columns=['Year Published'])

# Cleaning categorical columns
#### Identifying possible categorical columns

In [ ]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print("Categorical columns: ")
print(categorical_columns)

In [ ]:
# This is not correct because the Domains and Mechanics columns are not split yet
print(f"Number of unique values in the Domains column: {df['Domains'].nunique()}")
print(f"Number of unique values in the Mechanics column: {df['Mechanics'].nunique()}")

## Converting the Domains column
#### Splitting the Domains column

In [ ]:
def multi_hot_encoding_domains(df_to_use):
    df_to_use['Domains'] = df_to_use['Domains'].fillna('Unknown')
    df_to_use['Domains_split'] = df_to_use['Domains'].str.split(', ')

    all_domains_multi_encoding = df_to_use['Domains_split'].explode().unique()
    all_domains_multi_encoding = [domain for domain in all_domains_multi_encoding if domain != '']

    for domain in all_domains_multi_encoding:
        df_to_use[domain] = df_to_use['Domains_split'].apply(lambda x: 1 if domain in x else 0)

    if 'Unknown' in df_to_use.columns:
        df_to_use['Domains'] = df_to_use['Domains'].replace('Unknown', np.nan)
        df_to_use = df_to_use.drop(columns=['Unknown'])
        all_domains_multi_encoding.remove('Unknown')
    df_to_use = df_to_use.drop(columns=['Domains_split'])
    return df_to_use, all_domains_multi_encoding

In [ ]:
df, all_domains = multi_hot_encoding_domains(df)

## Converting the Mechanics column
### Custom mechanics categories
#### Converting the mechanics column into mapped clusters

In [ ]:
df_custom = df.copy()
df_custom['Mechanics_Categories'] = df['Mechanics'].apply(map_mechanics_to_categories)

#### Splitting the mechanics categories

In [ ]:
def multi_hot_encoding_mechanics(df_to_use):
    df_to_use['Mechanics_Categories'] = df_to_use['Mechanics_Categories'].fillna('Unknown')
    df_to_use['Mechanics_Categories_split'] = df_to_use['Mechanics_Categories'].str.split(', ')

    all_mechanics_multi_encoding = df_to_use['Mechanics_Categories_split'].explode().unique()
    all_mechanics_multi_encoding = [mechanic for mechanic in all_mechanics_multi_encoding if mechanic != '']

    for mechanic in all_mechanics_multi_encoding:
        df_to_use[mechanic] = df_to_use['Mechanics_Categories_split'].apply(lambda x: 1 if mechanic in x else 0)

    if 'Unknown' in df_to_use.columns:
        df_to_use['Mechanics_Categories'] = df_to_use['Mechanics_Categories'].replace('Unknown', np.nan)
        df_to_use = df_to_use.drop(columns=['Unknown'])
        all_mechanics_multi_encoding.remove('Unknown')
    df_to_use = df_to_use.drop(columns=['Mechanics_Categories_split'])
    return df_to_use, all_mechanics_multi_encoding

In [ ]:
df_custom, all_mechanics = multi_hot_encoding_mechanics(df_custom)

In [ ]:
df_custom.to_excel('./data/BGG_Cleaned_Data_Set_Mechanics_Imputing_Custom.xlsx', index=False)

### Self generated clusters for the mechanics column
#### Setting up the used models and the total list of mechanics

In [ ]:
df_embeddings = df.copy()
embedding_models = [
    'all-MiniLM-L6-v2',
    'all-mpnet-base-v2',
    'paraphrase-MiniLM-L6-v2',
    'multi-qa-MiniLM-L6-cos-v1',
    'all-distilroberta-v1',
    'all-roberta-large-v1',
    'stsb-roberta-large'
]

all_mechanics_embedding = set()
for mechanics in df_embeddings['Mechanics']:
    if pd.notna(mechanics):
        all_mechanics_embedding.update([m.strip() for m in mechanics.split(',')])

all_mechanics_embedding = list(all_mechanics_embedding)
print(f"Total amount of mechanics: {len(all_mechanics_embedding)}")

#### Generating the embeddings

In [ ]:
mechanic_clusterer = MechanicClustering(all_mechanics_embedding, embedding_models=embedding_models)
results = mechanic_clusterer.cluster_and_name_items(max_representatives=35)

In [ ]:
df_embeddings = mechanic_clusterer.map_to_clusters(df_embeddings, 'Mechanics')

In [ ]:
print(results['coverage_rate'])
print(results['best_configuration'])
df_embeddings

#### Splitting the generated embeddings and removing all the unnecessary columns

In [ ]:
# Splitting the Mechanics_Categories column into a list of categories
# Placeholder for the missing values
df_embeddings['mechanic_cluster'] = df_embeddings['mechanic_cluster'].fillna('Unknown')
df_embeddings['mechanic_cluster_split'] = df_embeddings['mechanic_cluster'].str.split(', ')

# Multi-hot encoding the Mechanics_Categories column
all_categories_embeddings = df_embeddings['mechanic_cluster_split'].explode().unique()
all_categories_embeddings = [category for category in all_categories_embeddings if category != '']

for category in all_categories_embeddings:
    df_embeddings[category] = df_embeddings['mechanic_cluster_split'].apply(lambda x: 1 if category in x else 0)

# Restoring NaN values in the original Mechanics_Categories column
df_embeddings['mechanic_cluster'] = df_embeddings['mechanic_cluster'].replace('Unknown', np.nan)
df_embeddings = df_embeddings.drop(columns=['Unknown', 'mechanic_cluster_split', 'Mechanics'])

In [ ]:
df_embeddings.to_excel('./data/BGG_Cleaned_Data_Set_Mechanics_Imputing_Embedded.xlsx', index=False)

## Imputing missing values
#### Checking for the columns with missing values

In [ ]:
df_custom.isna().sum().sort_values(ascending=False)

#### Imputing missing values in the Play Time column

In [ ]:
# Using median imputation because of the presence of outliers
df_custom['Play Time'] = df_custom['Play Time'].fillna(df_custom['Play Time'].median())

#### Imputing missing values in the Rating Average column

In [ ]:
# Using Mean imputation because of the normal distribution
df_custom['Rating Average'] = df_custom['Rating Average'].fillna(df_custom['Rating Average'].mean())

#### Imputing the missing values in the Domains columns

In [ ]:
def multi_hot_encoding_to_string_domains(row):
    domains = []
    for domain in all_domains:
        if row[domain] == 1:
            domains.append(domain)
    return ', '.join(domains) if domains else np.nan

In [ ]:
needed_features_domains = ['Complexity Average', 'Min Age', 'Play Time', 'Max Players', 'Users Rated', 'Owned Users']
missing_domains = df_custom[df_custom['Domains'].isna()]

In [ ]:
domain_imputing_results = domain_imputing_model.predict(missing_domains[needed_features_domains])
domain_imputing_results = pd.DataFrame(domain_imputing_results, columns=all_domains)

In [ ]:
missing_domains.loc[:, 'Domains'] = pd.DataFrame(domain_imputing_results).apply(multi_hot_encoding_to_string_domains, axis=1)

In [ ]:
df_custom.loc[missing_domains.index, 'Domains'] = missing_domains['Domains']

#### Imputing the missing values in the mechanics columns using default domain imputation

In [ ]:
remaining_missing_domains = df_custom[df_custom['Domains'].isna()]

In [ ]:
probability_domains = domain_imputing_model.predict(remaining_missing_domains[needed_features_domains])
predicted_probabilities_df = pd.DataFrame(probability_domains, columns=all_domains, index=remaining_missing_domains.index)
predicted_domains = predicted_probabilities_df.idxmax(axis=1)

In [ ]:
df_custom.loc[remaining_missing_domains.index, 'Domains'] = predicted_domains
print(f"Number of missing values in the Domains column: {df_custom['Domains'].isna().sum()}")

In [ ]:
df_custom.drop(columns=all_domains, inplace=True)
df_custom = multi_hot_encoding_domains(df_custom)[0]

In [ ]:
df_custom.to_excel('./data/BGG_Cleaned_Data_Set_Mechanics_Imputing_Custom_Imputed_Domains.xlsx', index=False)

#### Imputing the missing values in the Mechanics_Categories column using a custom imputation model

In [ ]:
all_mechanics_categories = list(df_custom['Mechanics_Categories'].str.split(', ').explode().unique())
all_mechanics_categories = [item for item in all_mechanics_categories if not (isinstance(item, float) and math.isnan(item))]

def multi_hot_encoding_to_string_mechanics(row):
    mechanics_list = []
    for mechanic in all_mechanics_categories:
        if row[mechanic] == 1:
            mechanics_list.append(mechanic)
    return ', '.join(mechanics_list) if mechanics else np.nan

In [ ]:
needed_features_mechanics = ['Strategy Games', 'Thematic Games', 'Wargames', 'Family Games',
       'Customizable Games', 'Children\'s Games', 'Abstract Games',
       'Party Games', 'Complexity Average', 'Play Time', 'Max Players',
       'Min Age']
missing_mechanics = df_custom[df_custom['Mechanics_Categories'].isna()]

In [ ]:
mechanics_imputing_result = mechanics_imputing_model.predict(missing_mechanics[needed_features_mechanics])
mechanics_imputing_result = pd.DataFrame(mechanics_imputing_result.squeeze(), columns=all_mechanics_categories)

In [ ]:
missing_mechanics.loc[:, 'Mechanics_Categories'] = pd.DataFrame(mechanics_imputing_result).apply(multi_hot_encoding_to_string_mechanics, axis=1)

In [ ]:
df_custom.loc[missing_mechanics.index, 'Mechanics_Categories'] = missing_mechanics['Mechanics_Categories']

#### Imputing the missing values in the mechanics columns using default mechanics imputation

In [ ]:
remaining_missing_mechanics = df_custom[df_custom['Mechanics_Categories'].isna()]

In [ ]:
probability_mechanics = mechanics_imputing_model.predict(remaining_missing_mechanics[needed_features_mechanics])
predicted_probabilities_mechanics_df = pd.DataFrame(probability_mechanics.squeeze(), columns=all_mechanics_categories, index=remaining_missing_mechanics.index)
predicted_mechanics = predicted_probabilities_mechanics_df.idxmax(axis=1)

In [ ]:
df_custom.loc[remaining_missing_mechanics.index, 'Mechanics_Categories'] = predicted_mechanics
print(f"Number of missing values in the Mechanics_Categories column: {df_custom['Mechanics_Categories'].isna().sum()}")

In [ ]:
df_custom.drop(columns=all_mechanics_categories, inplace=True)
df_custom.drop(columns=['Mechanics'], inplace=True)
df_custom.dropna(subset=['Mechanics_Categories'], inplace=True)
df_custom = multi_hot_encoding_mechanics(df_custom)[0]

In [ ]:
df_custom.to_excel('./data/BGG_Complete_Cleaned_Dataset.xlsx', index=False)